In [10]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split

from Classes.CartTreeClassifier import CartTreeClassifier
from Classes.CartTreeTeacher import CartTreeTeacher

from Functions.metrics import *
from Functions.split_functions import *
from Functions.spreading_functions import *
from Functions.search_split_functions import *

from Functions.metrics import accuracy
from sys import setrecursionlimit
from imblearn.over_sampling import SMOTE
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler

from Classes.LayerGen import LayerGenerator
from Classes.Layer import Layer
from Classes.Perceptron import Perceptron
from Classes.PerceptronBWSearcher import PerceptronBWSearcher

from Functions.function_of_activation import sigmoid, dsigmoid
from Functions.activation_functions import activation_with_cach, standart_activation
from Functions.lossFunctions import euclideError
from Functions.errorFunctions import MSE


In [11]:
data = pd.read_csv('../Data/DataLaba2/neo_task_smoted.csv')
mm_scaler = MinMaxScaler()
data_scaled = mm_scaler.fit_transform(data)
data_scaled = pd.DataFrame(data_scaled, columns=data.columns)
data_scaled_x, hazardous = SMOTE().fit_resample(data_scaled.drop(['hazardous'], axis=1), data['hazardous'])
data_scaled_x = pd.DataFrame(data_scaled_x, columns=data.columns[data.columns != 'hazardous'])
X_train, X_test, y_train, y_test = train_test_split(data_scaled, hazardous, test_size=0.5, random_state=42)


In [12]:
layer_mass = LayerGenerator(sizes_of_input_layer=X_train.shape[1],
                            sizes_of_hidden_layers=[X_train.shape[1] // 2, X_train.shape[1] // 3],
                            sizes_of_output_layer=1,
                            activation_functions=[sigmoid,
                                                  sigmoid,
                                                  sigmoid,
                                                  sigmoid],
                            dactivation_functions=[dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid,
                                                   dsigmoid],
                            function_of_generation=lambda a, b: np.random.rand(a, b)).generate()
layer_mass


In [13]:
def batch_spliter(x, y, size, num):
    batches_x = list()
    batches_y = list()
    for _ in range(num):
        args = np.arange(x.shape[0])
        np.random.shuffle(args)
        batches_x.append(x[args][:size])
        batches_y.append(y[args][:size])
    return zip(batches_x, batches_y)


In [14]:
def MSE(predicted, real):
    return np.sum((real-predicted)**2, axis=1)/real.shape[0]


In [15]:
batch_split = list(batch_spliter(X_train[:50].to_numpy(), y_train[:50].to_numpy(), 10, 10))
batch_split


[(array([[3.64125835e-04, 2.59016470e-01, 5.49847114e-01, 7.13391740e-01,
          0.00000000e+00],
         [3.38660836e-02, 2.96704015e-01, 2.70912770e-02, 3.06640813e-01,
          1.00000000e+00],
         [5.64704079e-04, 1.53028201e-01, 7.39863349e-02, 6.75010430e-01,
          0.00000000e+00],
         [1.49121143e-02, 5.15341874e-01, 8.58273946e-01, 3.80892783e-01,
          1.00000000e+00],
         [6.44694678e-05, 6.46385961e-02, 1.81663341e-02, 8.53984147e-01,
          0.00000000e+00],
         [1.37394494e-04, 1.16308856e-01, 5.87067128e-01, 7.95577806e-01,
          0.00000000e+00],
         [3.34653745e-03, 2.32008584e-01, 1.57611532e-01, 5.06883605e-01,
          1.00000000e+00],
         [3.24028161e-03, 4.40234906e-01, 1.26905277e-01, 5.18839633e-01,
          1.00000000e+00],
         [2.77651905e-03, 1.92189463e-01, 2.50238250e-01, 5.32749270e-01,
          1.00000000e+00],
         [3.87582209e-04, 2.41363636e-01, 8.36840794e-01, 7.07968294e-01,
          0.00000

In [16]:
perceptron = PerceptronBWSearcher(layers=layer_mass, 
                         activation_function=activation_with_cach, 
                        dactivation_function=None, 
                        batch_spliter_function=lambda x, y: batch_spliter(x, y, 10, 10), 
                                  error_function=MSE,
                        n_steps=10)


In [17]:
perceptron.search(X_train[:50].to_numpy(), y_train[:50].to_numpy())


[0.28813218 0.2877299  0.2876055  0.28782464 0.28819104 0.28821152
 0.28783185 0.28775842 0.28827294 0.28817264]
[0.28816084 0.28817632 0.28813896 0.28817264 0.28782623 0.28776131
 0.28788723 0.2876055  0.28824133 0.28772061]
[0.28773614 0.28817402 0.28824325 0.28817264 0.2877299  0.28779291
 0.28813218 0.28788723 0.28817632 0.28767382]
[0.3266737  0.32638977 0.32722643 0.3265832  0.32684881 0.32725635
 0.32662585 0.32651899 0.32720922 0.32735501]
[0.32662585 0.32683738 0.32735915 0.32684881 0.3265642  0.32735501
 0.32726148 0.3265971  0.32681651 0.32720922]
[0.36576274 0.3656221  0.36561886 0.36630005 0.36545188 0.36577374
 0.36639189 0.36625576 0.36517405 0.36578905]
[0.3655503  0.36542264 0.36548453 0.36564737 0.36625576 0.36629467
 0.36577374 0.36578276 0.36578905 0.36646677]
[0.24908439 0.24912905 0.2489283  0.24913283 0.24896227 0.24891111
 0.24909768 0.24913055 0.24889537 0.24913149]
[0.32719736 0.32657828 0.3266737  0.32672175 0.32730705 0.32726148
 0.32735211 0.32651899 0.3266